# Linear Algebra Overview

In [2]:
%load ../../rapaio-bootstrap.ipynb

Adding dependency io.github.padreati:rapaio-lib:7.0.1
Solving dependencies
Resolved artifacts count: 1
Add to classpath: /home/ati/work/rapaio-jupyter-kernel/target/mima_cache/io/github/padreati/rapaio-lib/7.0.1/rapaio-lib-7.0.1.jar


`DArrays` are multi-dimensional arrays which stores data elements of the same type indexed by dimensions. Current implementation offers only stride dense vector arrays, but the design allows future implementations like sparse arrays.

DArrays offers many operations, besides standard manipulation data tools, there are implemented also some non trivial operations and also matrix decompositions and linear solvers. There are implementations for four numerical data types: `byte`, `int`, `float` and `double`. 

## DArrayManager

Working with darrays starts with having a `DArrayManager`. A DArrayManager handles all the internals which are required for an implementation of DArrays. Even if there is a single implementation of DArrays, which works with standard Java arrays and uses stride algebra and simd vectorization, multiple implementations are possible. One example is to use MemorySegments when simd instructions will be able to work with indexed operations.

DArray manager describes which storage factory to be used, which implementation of the operations, and other various parameters.

One can create darrays with this tensor manager using also `DArrays` class. However it is advisable to use a DArrayManager every time since this will maintain code compatbility in the future when new DArray implementations will be available. Switching to other implementations will require a single line of code change, where you choose the DArrayManager implementation.

In [16]:
DArrayManager dm = DArrayManager.base();
DType dt = DType.DOUBLE;

## Shape of a DArray

DArray elements are indexed by dimensions. You can think of a darray as a hypercube of data elements with one element in each position of the hypercube. A darray can have 0, 1, 2 or more dimensions. Each dimension have a dimension size. The dimensions and the size of each dimension is described by a `Shape` object.

The total number of elements is given by the product of all dimension sizes, or 1 if there is no dimension.

In [4]:
// a shape with no dimensions
display(Shape.of());

// a shape with 1 dimension / a vector of size 1
display(Shape.of(1));

// Both shapes are of size 1, but they are different shapes
display(Shape.of().size());
display(Shape.of(1).size());

Shape: []

Shape: [1]

1

1

fce0e044-c1b6-4c45-937b-ef2f00a20ca8

## DArray creation

A darray with no dimensions is a scalar and can be created with:

In [17]:
var scalar = dm.scalar(dt, 1.);
display(scalar.shape());
display(scalar)

Shape: []

BaseStride{DOUBLE,[],0,[]}
 1  


cde09ed4-402e-4221-9349-040374fe9731

A DArray with a single dimension is a vector, with two dimensions it is called a matrix and in general with more than two dimensions it is called a tensor. 

In [18]:
dm.seq(DType.FLOAT, Shape.of(100)).printFullContent()

[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 ] 


Since the default darray manager uses Java arrays as data storage, it is possible to wrap an array into a darray of appropriate type and use darray operations to change values in the array. For example we will create an array of double values and a wrapper darray around it. 
Using darray methods we can change the values from the array.

In [19]:
double[] array = new double[] {1., 2., 3.};
// the default tensor manager wraps a double array if we request a Tensor<Double>
var t = dm.stride(dt, array);
t.log1p_(); // call the in-place log1p operation
Arrays.stream(array).forEach(System.out::println);

0.6931471805599453
1.0986122886681096
1.3862943611198906


The same thing happens for other value types (from the supperted ones which are: `double`,`float`,`int` and `byte`)

In [20]:
float[] floatArray = new float[] {1.f, 2, 3, 4};
var floatTensor = dm.stride(DType.FLOAT, floatArray);
floatTensor.sqr_();
for(float f : floatArray) {
    System.out.println(f);
}

1.0
4.0
9.0
16.0


Using a different type will not wrap the array, but will copy the data into a new darray.

In [21]:
var doubleArray = dm.stride(dt, floatArray);
doubleArray.sqrt_();
for(int i=0; i<floatArray.length; i++) {
    System.out.printf("original value at index[%d]: %f, corresponding darray value: %f%n", 
                      i, floatArray[i], doubleArray.get(i));
}
// we can see that the values remain unchanged

original value at index[0]: 1.000000, corresponding darray value: 1.000000
original value at index[1]: 4.000000, corresponding darray value: 2.000000
original value at index[2]: 9.000000, corresponding darray value: 3.000000
original value at index[3]: 16.000000, corresponding darray value: 4.000000


The are also other constructors. 

In [22]:
var m = dm.eye(dt, 3);
display("m:");
display(m);

Random random = new Random(42);
var u = dm.random(dt, Shape.of(3), random);
display("u:");
display(u);
display("vector inner product between u and third row of m, which basically is the third element of u:");
display(u.inner(m.selsq(0, 2)));
display(u.get(2))

m:

BaseStride{DOUBLE,[3, 3],0,[3, 1]}
[[ 1 0 0 ]  
 [ 0 1 0 ]  
 [ 0 0 1 ]] 


u:

BaseStride{DOUBLE,[3],0,[1]}
[ 0.6054611363173034 0.47673188281383294 -0.49948335406718436 ] 


vector inner product between u and third row of m, which basically is the third element of u:

-0.49948335406718436

-0.49948335406718436

2f14bf10-8a52-4917-8659-e8e49696ec00

Since `m` is the identity matrix, it's transpose is also identity, 
matrix multiplication between the two produces also identity and 
the determinant of `m` should be `1`:

In [23]:
var mtm = m.t_().mm(m);
display("text/markdown", "$m^T m = I$");
mtm.printContent();
mtm.lu().det();

$m^T m = I$

[[ 1 0 0 ]  
 [ 0 1 0 ]  
 [ 0 0 1 ]] 


1.0

## Stride arrays

DArray base implementation uses stride array algebra. In short, all the elements of the darrays are stored in a storage which is a contiguous array. A darray is a view over the storage which defines a stride arithmetic. If operations allows to use the same storage (it is possible to perform the transformation manipulating only shape, offset and stride information), than it will create a new view. This allows fast operations if possible.

Let's build an array which contains a sequence of numbers.

In [24]:
var x = dm.seq(dt, Shape.of(2,3,4));
x

BaseStride{DOUBLE,[2, 3, 4],0,[12, 4, 1]}
[[[  0  1  2  3 ]   
  [  4  5  6  7 ]   
  [  8  9 10 11 ]]  
 [[ 12 13 14 15 ]   
  [ 16 17 18 19 ]   
  [ 20 21 22 23 ]]] 


Let's dive a little bit into the description of the darray. `BaseStride` is the class which implements the darray, which means it uses stride arithmetic to index the values from storage. `FLOAT` is the data type of the elements, in this case being `float` value type from Java.

Next we have some numeric information. `[2,3,4]` is the shape. This is to be expected, since we specified that shape when we created the darray. This means we have a darray with three dimensions, first axis of size `2`, second axis of size `3` and the third axis of size `4`.

The next information is the number `0`. This is the offset. The offset for a strided array is a position in the contiguous storage where the first element of the darray is stored. Sometimes this offset is not zero, and we will provide and example soon.

The last numeric information is `[12,4,1]`. These are the strides. We have a stride for each dimension in the darray. The meaning of stride is the following: *how many elements have to be skipped to see the next element in the given dimension*. The stride and offset gives a way to compute the position of an element in the storage, given an index for that element.

$$ptr[i,j,k] = offset + i*stride[0] + j*stride[1] + k * stride[2]$$

The above formula is valid for three dimensions, of course. In general for `D` dimensions we have:

$$ptr[i_0,i_1,..,i_{D-1}] = offset + \sum_{j=0}^{D-1} i_j * stride[j]$$

`x` is a 3 dimensional array. The first dimension has size `2`. This allows the interpretation of `x` as being an array of 2 matrices of shape `[3,4]`. We can select an element based on an axis and index using `sel`. Furthermore, `selsq`, which is an abbreviation of *select and squeeze* removes the given axis from the darray.

In [25]:
// first matrix
display(x.selsq(0,0));
display(x.selsq(0,1));

BaseStride{DOUBLE,[3, 4],0,[4, 1]}
[[ 0 1  2  3 ]  
 [ 4 5  6  7 ]  
 [ 8 9 10 11 ]] 


BaseStride{DOUBLE,[3, 4],12,[4, 1]}
[[ 12 13 14 15 ]  
 [ 16 17 18 19 ]  
 [ 20 21 22 23 ]] 


7a8541bc-8dfe-4714-8cff-426c0e09ee72

Notice the offset of the second darray, which is `12`. The two darrays have the same shapes and strides, which means their layout in memory is the same, but their offset is different. This is because `x` is stored in C order (the default order in storage, but that can be changed also), which means it first stores the elements of the first matrix at offset `0` and after that it stores the elements of the second matrix at offset `12`, which is the same value as the number of elements of the first darray.

Multiple other operations on the layout creates different views on the same storage. General transposition, for example, creates the following darray.

In [26]:
x.t_()

BaseStride{DOUBLE,[4, 3, 2],0,[1, 4, 12]}
[[[  0 12 ]   
  [  4 16 ]   
  [  8 20 ]]  
 [[  1 13 ]   
  [  5 17 ]   
  [  9 21 ]]  
 [[  2 14 ]   
  [  6 18 ]   
  [ 10 22 ]]  
 [[  3 15 ]   
  [  7 19 ]   
  [ 11 23 ]]] 


Notice how the shape dimensions are reversed, the same offset, but strides are different, since there are different dimension sizes. To check if two darrays have the same storage one have to compare the storage instances.

In [27]:
display(x.storage());
display(x.t_().storage());
display(x.storage() == x.t_().storage());

rapaio.darray.storage.array.DoubleArrayStorage@7d65e2c7

rapaio.darray.storage.array.DoubleArrayStorage@7d65e2c7

true

6abe27d1-e2d7-494e-a8f0-b821c23aa7db

**Note on syntax:**
Some methods have two variants, the first one named `operation` and the second one `operation_`, which is the same name ended with underscore. When the method name ends with underscore the operation works on the same storage as the original darray, when there is no ending underscore a new copy is created. Not all operations allows this syntax, since some are not able to work on the same storage and some others works only on the same storage. 

When this syntax is used, both scenarios are implemented. This is valid for the transpose operation also. Notice the storage instance:

In [28]:
display(x.storage());
display(x.t_().storage());
display(x.t().storage());

rapaio.darray.storage.array.DoubleArrayStorage@7d65e2c7

rapaio.darray.storage.array.DoubleArrayStorage@7d65e2c7

rapaio.darray.storage.array.DoubleArrayStorage@f1ca99e

e1977075-a1a3-4683-b674-e3975d9512b7

## Data iterators

The elements of a darray can be accessed in multiple ways. One way is based on indexes. The index of an element is an array of semi-positive integer numbers, each less than its corresponding dimension. Thus the index of an element describes the order of that element in each dimension.

In [30]:
dt = DType.FLOAT;
var x = dm.random(dt, Shape.of(4,5), random);
x

BaseStride{FLOAT,[4, 5],0,[5, 1]}
[[  0.682409405708313   -1.860346794128418   -0.3640243411064148   0.907001256942749   -0.2078128606081009 ]  
 [  1.9437248706817627   0.5633633732795715  -0.04871172457933426 -0.5485519170761108   1.6434990167617798 ]  
 [  0.9172410368919373   0.3495040833950043  -0.3341809809207916  -0.35777759552001953 -0.1645195335149765 ]  
 [ -0.10719511657953262 -0.06851354241371155 -0.07772722095251083  0.9436370730400085  -1.0320192575454712 ]] 


We have a matrix of `4` rows and `5` columns. To access the element from second row and third column we can use `get` methods:

In [32]:
display(x.get(1,2));
// if we want directly the value type
display(x.getFloat(1,2));
// we can de a cast, also, if it is useful, notice that 0 is the rounded value of the original one
display(x.getInt(1,2));

-0.048711725

-0.048711725

0

6d62c01f-11d2-4933-aea0-e7d8d8e4f4a4

To iterate on the elements using indexes one can use the following iterator:

In [33]:
var it = x.iterator();
while(it.hasNext()) {
    System.out.print(it.next() + ", ");
}

0.6824094, -1.8603468, -0.36402434, 0.90700126, -0.20781286, 1.9437249, 0.5633634, -0.048711725, -0.5485519, 1.643499, 0.91724104, 0.34950408, -0.33418098, -0.3577776, -0.16451953, -0.10719512, -0.06851354, -0.07772722, 0.9436371, -1.0320193, 

The default order is `Order.C` C-style order, which means last dimensions loops faster. In the case of the matrix we iterate first on rows, and after that, in the inner loop, we iterate over columns. So, in C order, columns loops faster than rows. 

One can use also other orders, like `Order.F` Fortran-style order, which means first dimensions loops faster. There is also `Order.S` which is the storage order, the best possible order in which elements can be iterated so that accessing them is cache friendly. Also, `Order.A` means automatic order, which can be determined by an algorithm. We also have `Order.defaultOrder()` which is the configured default order, in our case being C.

In [34]:
var it = x.iterator(Order.F);
while(it.hasNext()) {
    System.out.print(it.next()+", ");
}

0.6824094, 1.9437249, 0.91724104, -0.10719512, -1.8603468, 0.5633634, 0.34950408, -0.06851354, -0.36402434, -0.048711725, -0.33418098, -0.07772722, 0.90700126, -0.5485519, -0.3577776, 0.9436371, -0.20781286, 1.643499, -0.16451953, -1.0320193, 

Another kind of iterator is to iterate by pointer. The pointer of an element is basically the position of that element in the underlying storage. The advantage of using pointer elements is that it can be faster since some optimizations are realized if possible. To access elements based on their pointer value you can used `ptrGet` methods.

In [35]:
var pit = x.ptrIterator();
while(pit.hasNext()) {
    System.out.print(x.ptrGet(pit.next()) + ", ");
}

0.6824094, -1.8603468, -0.36402434, 0.90700126, -0.20781286, 1.9437249, 0.5633634, -0.048711725, -0.5485519, 1.643499, 0.91724104, 0.34950408, -0.33418098, -0.3577776, -0.16451953, -0.10719512, -0.06851354, -0.07772722, 0.9436371, -1.0320193, 

## Operations on shapes

In general we can consider that a darray is an organized indexed view over some data elements which are lies into a storage. Many operations on those objects changes only this organization of data. Some of the operations are able to create a new view over the same data and for some other this is impossible, and a new storage with copied data is necessary. 

The idea of multidimensional array of elements can be seen in a recursive fashion. For example a darray in three dimensions can be understood as an indexed array of darrays of 2 dimensions, which in turn can be seen as arrays of unidimensional objects, which in a final turn can be seen as an indexed array of individual values, or scalars. As we can see this is a matter of organization, and this kind of operations helps us to give a proper form more usefull for understanding, for computation or for other reasons.

In [57]:
var x = dm.seq(dt, Shape.of(2,3,2));
x

BaseStride{FLOAT,[2, 3, 2],0,[6, 2, 1]}
[[[  0  1 ]   
  [  2  3 ]   
  [  4  5 ]]  
 [[  6  7 ]   
  [  8  9 ]   
  [ 10 11 ]]] 


In [62]:
// split into pieces of the same size on the first dimension
x.chunk(0, false, 1);

[BaseStride{FLOAT,[3, 2],0,[2, 1]}
[[ 0 1 ]  
 [ 2 3 ]  
 [ 4 5 ]] 
, BaseStride{FLOAT,[3, 2],6,[2, 1]}
[[  6  7 ]  
 [  8  9 ]  
 [ 10 11 ]] 
]

In [72]:
// split into pieces of the same size on first and last dimension
List<DArray<Float>> list = x.chunkAll(false,new int[]{1,3,1});
list.forEach(da->display(da.squeeze(0, 2)));

BaseStride{FLOAT,[3],0,[2]}
[ 0 2 4 ] 


BaseStride{FLOAT,[3],1,[2]}
[ 1 3 5 ] 


BaseStride{FLOAT,[3],6,[2]}
[ 6 8 10 ] 


BaseStride{FLOAT,[3],7,[2]}
[ 7 9 11 ] 


In [73]:
// define some vectors
var a = dm.random(dt, Shape.of(3), random);
var b = dm.random(dt, Shape.of(3), random);
var c = dm.random(dt, Shape.of(3), random);
display(a);
display(b);
display(c);

BaseStride{FLOAT,[3],0,[1]}
[ -0.32171013951301575 -0.17304112017154694 -0.4335041344165802 ] 


BaseStride{FLOAT,[3],0,[1]}
[ -0.48131176829338074 -0.144036203622818 -1.3020426034927368 ] 


BaseStride{FLOAT,[3],0,[1]}
[ 0.6593151688575745 -0.950773298740387 0.8610353469848633 ] 


cc5a5b5c-8e6a-4f18-a5c5-1b14ae077790

In [83]:
// stack all of them as rows in a matrix. 
dm.stack(0, List.<DArray<Float>>of(a,b,c))

BaseStride{FLOAT,[3, 3],0,[3, 1]}
[[ -0.32171013951301575 -0.17304112017154694 -0.4335041344165802 ]  
 [ -0.48131176829338074 -0.144036203622818   -1.3020426034927368 ]  
 [  0.6593151688575745  -0.950773298740387    0.8610353469848633 ]] 


In [84]:
// stack all of them as columns in a matrix
dm.stack(1, List.<DArray<Float>>of(a,b,c))

BaseStride{FLOAT,[3, 3],0,[3, 1]}
[[ -0.32171013951301575 -0.48131176829338074  0.6593151688575745 ]  
 [ -0.17304112017154694 -0.144036203622818   -0.950773298740387  ]  
 [ -0.4335041344165802  -1.3020426034927368   0.8610353469848633 ]] 


In [85]:
// concatenate all elements along the single available dimension of the elements
dm.cat(0, List.<DArray<Float>>of(a,b,c))

BaseStride{FLOAT,[9],0,[1]}
[ -0.32171013951301575 -0.17304112017154694 -0.4335041344165802 -0.48131176829338074 -0.144036203622818 -1.3020426034927368 0.6593151688575745 -0.950773298740387 0.8610353469848633 ] 


## Elementwise operations

## Reduce operations

## Binary operations

## Broadcasting

## Matrix decompositions

Some fundamental matrix decompositions are available. Usually those can be found as operations with an abbreviated name. Some examples includes singluar value decomposition (`svd`), eigenvalue decomposition (`eig`), or QR decomposition `qr`.

In [36]:
var x = dm.random(dt, Shape.of(4,2), random);
display(x);

BaseStride{FLOAT,[4, 2],0,[2, 1]}
[[  0.9695566296577454  -0.09966190159320831 ]  
 [ -0.5811431407928467  -0.8561254143714905  ]  
 [ -0.9181374907493591   1.1062313318252563  ]  
 [ -0.03361855819821358 -0.19940824806690216 ]] 


e349cf03-69e0-4062-be5d-f30bcd1dfc89

In [37]:
// singular value decomposition
var u = x.svd().u();
var s = x.svd().s();
var v = x.svd().v();

display(u);
display(s);
display(v);

// reconstruction of x
display(u.mm(s).mm(v.t()));

display("rank of x:");
display(x.svd().rank());

BaseStride{FLOAT,[4, 2],0,[2, 1]}
[[  0.4795474410057068  -0.4799862802028656  ]  
 [  0.08935008198022842  0.8497121930122375  ]  
 [ -0.8703932762145996  -0.16638758778572083 ]  
 [  0.06683049350976944  0.14112931489944458 ]] 


BaseStride{FLOAT,[2, 2],0,[2, 1]}
[[ 1.635522723197937 0                 ]  
 [ 0                 1.205543041229248 ]] 


BaseStride{FLOAT,[2, 2],0,[2, 1]}
[[  0.7397739887237549 -0.6728554368019104 ]  
 [ -0.6728554368019104 -0.7397739887237549 ]] 


BaseStride{FLOAT,[4, 2],0,[2, 1]}
[[  0.9695565700531006  -0.0996619164943695  ]  
 [ -0.5811430811882019  -0.8561253547668457  ]  
 [ -0.9181374907493591   1.1062313318252563  ]  
 [ -0.03361855447292328 -0.19940824806690216 ]] 


rank of x:

2

fad95e59-5b59-4df9-a563-b59f9e465d1d

In [38]:
// QR decomposition
x.qr().q()

BaseStride{FLOAT,[4, 2],0,[2, 1]}
[[ -0.665600061416626     0.1316402405500412  ]  
 [  0.39895445108413696  -0.7555326819419861  ]  
 [  0.6303008794784546    0.6228885054588318  ]  
 [  0.023079121485352516 -0.15443818271160126 ]] 


In [39]:
// q is orthogonal
var q = x.qr().q();
display("norm of the first vector column:");
display(q.selsq(1,0).inner(q.selsq(1,0)));
display("inner product of the two column vectors:");
display(q.selsq(1,0).inner(q.selsq(1,1)));

norm of the first vector column:

0.99999994

inner product of the two column vectors:

4.656613E-10

1fcb2e52-84a3-4b8d-991f-fa0dfc7194ed

In [40]:
var r = x.qr().r();
// basic reconstruction
display(x);
display(q.mm(r));

BaseStride{FLOAT,[4, 2],0,[2, 1]}
[[  0.9695566296577454  -0.09966190159320831 ]  
 [ -0.5811431407928467  -0.8561254143714905  ]  
 [ -0.9181374907493591   1.1062313318252563  ]  
 [ -0.03361855819821358 -0.19940824806690216 ]] 


BaseStride{FLOAT,[4, 2],0,[2, 1]}
[[  0.9695565700531006  -0.09966188669204712 ]  
 [ -0.5811431407928467  -0.8561254739761353  ]  
 [ -0.9181374907493591   1.106231451034546   ]  
 [ -0.03361855819821358 -0.19940824806690216 ]] 


89ea25d3-15ea-4b73-a76a-5c4924531fce

In [41]:
// compute inverse of a matrix using QR decomposition
var xinv = x.qr().inv();
display(xinv);
display(xinv.mm(x));

BaseStride{FLOAT,[2, 4],0,[4, 1]}
[[ 0.48480424284935    -0.433839350938797  -0.3008265793323517 -0.0485406331717968 ]  
 [ 0.09725435823202133 -0.5581792593002319  0.4601832330226898 -0.1140972450375557 ]] 


BaseStride{FLOAT,[2, 2],0,[2, 1]}
[[  1                        -0.0000000046566128730773926 ]  
 [ -0.0000000421423465013504  0.9999998807907104          ]] 


a5c06ce4-1671-4f2d-924d-8838274cf57d

In [43]:
var x = dm.random(dt, Shape.of(4, 4), random);
x

BaseStride{FLOAT,[4, 4],0,[4, 1]}
[[  0.10020556300878525  0.3454208970069885 -1.1442604064941406  -0.006700537167489529 ]  
 [ -1.9218206405639648  -1.7930243015289307 -0.04050852730870247  0.398999959230423    ]  
 [ -0.2643210291862488   1.5562230348587036 -0.2933931052684784   0.3751673102378845   ]  
 [  0.7403811812400818   1.2483185529708862  0.25033804774284363  1.9779791831970215   ]] 


In [44]:
var y = dm.random(dt, Shape.of(4), random);
y

BaseStride{FLOAT,[4],0,[1]}
[ -1.424954891204834 0.7558175325393677 -0.5863436460494995 1.2770365476608276 ] 


In [45]:
x.mul(y)

BaseStride{FLOAT,[4, 4],0,[4, 1]}
[[ -0.14278841018676758  0.2610751688480377  0.6709297895431519   -0.008556830696761608 ]  
 [  2.7385077476501465  -1.3551992177963257  0.023751918226480484  0.5095375180244446   ]  
 [  0.3766455352306366   1.1762206554412842  0.17202918231487274   0.47910237312316895  ]  
 [ -1.0550098419189453   0.9435010552406311 -0.14678412675857544   2.525951623916626    ]] 


In [46]:
x.mul(y.stretch(1)) 

BaseStride{FLOAT,[4, 4],0,[4, 1]}
[[ -0.14278841018676758 -0.49220919609069824  1.6305195093154907   0.009547962807118893 ]  
 [ -1.4525457620620728  -1.3551992177963257  -0.03061705455183983  0.30157116055488586  ]  
 [  0.15498295426368713 -0.9124814867973328   0.17202918231487274 -0.21997696161270142  ]  
 [  0.9454938173294067   1.5941483974456787   0.3196908235549927   2.525951623916626    ]] 


In [47]:
x.mul(y.stretch(0))

BaseStride{FLOAT,[4, 4],0,[4, 1]}
[[ -0.14278841018676758  0.2610751688480377  0.6709297895431519   -0.008556830696761608 ]  
 [  2.7385077476501465  -1.3551992177963257  0.023751918226480484  0.5095375180244446   ]  
 [  0.3766455352306366   1.1762206554412842  0.17202918231487274   0.47910237312316895  ]  
 [ -1.0550098419189453   0.9435010552406311 -0.14678412675857544   2.525951623916626    ]] 


## Copy, cast, concatenate, split